# Distributional Forecasting in Electiricity Markets: <br>Prediction Interval Averaging vs Quantile Regression Averaging

### References
- Christoffersen, P. F. (1998). Evaluating interval forecasts. International economic review, 841-862.
- Gaba, A., Tsetlin, I., & Winkler, R. L. (2017). Combining interval forecasts. Decision Analysis, 14(1), 1-20.
- Nowotarski, J., & Weron, R. (2018). Recent advances in electricity price forecasting: A review of probabilistic forecasting. Renewable and Sustainable Energy Reviews, 81, 1548-1568.
- Uniejewski, B., Nowotarski, J., & Weron, R. (2016). Automated variable selection and shrinkage for day-ahead electricity price forecasting. Energies, 9(8), 621.
- Weron, R., & Misiorek, A. (2008). Forecasting spot electricity prices: A comparison of parametric and semiparametric time series models. International journal of forecasting, 24(4), 744-763.

#### Notes:
- from **2018-04-02** to 2019-03-31 individual model training
- from **2019-04-01** to 2020-03-29 individual model forecasting & PI training
- from **2020-03-30** to 2021-03-28 individual model forecasting & PI forecasting

In [1]:
import pandas as pd
import numpy as np
import re
from code.holiday import get_holiday
from code.forecast import get_forecast_AR, get_forecast_QRA, get_naive_forecast, PI_combinations
from code.evaluation import get_PI_from_distribution, Christoffersen_scores, pinball_loss, winkler_score, DM_test
from statsmodels.tools.sm_exceptions import ValueWarning
import matplotlib.pyplot as plt
from scipy import stats
import plotly.graph_objects as go
import os
import warnings

# ignore the frequency error in ARIMA function
warnings.filterwarnings('ignore', category=ValueWarning)

### Import data

In [2]:
price_df_raw = pd.read_csv('data/MCP-25032018-28032021.csv')
load_df_raw = pd.read_csv('data/LoadForecast-01042018-28032021.csv')
# after 03/10/2018 -> permanent +03 gmt

preprocess = lambda df: df.set_index( 
    # merge date and time to get datetime and set index
    pd.to_datetime(
        df['Date'] + " " + df['Hour'],
        dayfirst=True
    )
) \
.drop(columns=['Date', 'Hour']) \
.apply(
    # remove decimal seperator ","
    lambda col: col.astype(str).str.replace(',','').astype(float)
) \
.tz_localize('Europe/Istanbul') # add timezone info

price_df = preprocess(price_df_raw)
price_df.rename(columns={x: re.findall('\((.*)/', x)[0] for x in price_df.columns}, inplace=True)

load_df = preprocess(load_df_raw).rename(columns={'Load Forecast (MWh)':'Load_Forecast_MWh'})

### Convert data to hourly shape

In [3]:
# Hourly Dataframes
## concatenate each hour's dataset vertically
get_hourly_df = lambda df, column: df.groupby(df.index.hour)[column].apply(
    lambda col: col.to_dict()).to_frame().dropna()

hourly_load_df = get_hourly_df(load_df, 'Load_Forecast_MWh')
hourly_price_df = pd.concat(
    [ get_hourly_df(price_df, col) for col in price_df ],
    axis=1
)
hourly_holiday_df = get_hourly_df(get_holiday(price_df), 'Holiday')
hourly_min_price_l1 = pd.concat(
    [get_hourly_df(price_df.resample('D').transform('min').shift(24), col) for col in price_df],
    axis=1
)
hourly_min_price_l1.rename(columns={x: f"{x}_min_l1" for x in hourly_min_price_l1.columns}, inplace=True)

hourly_weekday_df = hourly_price_df.assign(
    Saturday = lambda df: (df.index.get_level_values(1).day_of_week == 5) * 1,
    Sunday = lambda df: (df.index.get_level_values(1).day_of_week == 6) * 1,
    Monday = lambda df: (df.index.get_level_values(1).day_of_week == 0) * 1,
).filter(regex='day')

### Choose currency and merge the series

In [4]:
currency = 'USD' # 'MCP (TL/MWh), PTF (USD/MWh), PTF (EUR/MWh)
main_df = pd.concat([
    hourly_price_df[currency].to_frame(name='Price_MWh').groupby(level=0, axis=0).apply(lambda hour: hour.diff(1)),
    hourly_load_df.groupby(level=0, axis=0).apply(lambda hour: hour.diff(1)),
    hourly_min_price_l1.filter(regex=currency).rename(columns={f"{currency}_min_l1": "Price_MWh_min_l1"})
    .groupby(level=0, axis=0).apply(lambda hour: hour.diff(1)),
    hourly_holiday_df,
    hourly_weekday_df
    ], axis=1
)

price_df_final = price_df[currency].to_frame(name='Price_MWh')

In [5]:
price_df_final.to_parquet('./data/Price_df_processed.parquet')

# Individual Models
Three different models are employed; ARX (Autoregresive Model with exogenous variables), mARX (multi-day ARX) and TARX (Threshold Autoregressive model with exogenous variables). ARX and mARX models follows the the structure in Nowotarski & Weron(2018), and TARX model follows the structure explained in Weron & Misiorek (2008). Models with lags 1 to 7 are also tried to be regularized using _Lasso_ (least absolute shrinkage and selection operator), its implementation is inspired by Uniejewski et al. (2016).

Each Model is constructed seperately for each hour due to correlative behavior of prices in a day. Except models with lasso, all models are estimated using statsmodels' ARIMA function with yule walker equations as estimator.


### Notation:
$p_{d,h}$: the at $h^{th}$ hour in day $d$.<br>
$p_{d-1,h}^{min}$: minimum price in the previous day.<br>
$load_{d,h}$: Load forecast for $h^{th}$ hour in day $d$.<br>
$D_{day, h}$: Dummy for $day \in\{Saturday, Sunday, Monday, Holiday\}$ where holiday is national holidays in Turkey.<br>
**Note**: Due to stationarity concerns; price, price lags, minimum prices and load forecasts are differenced by 1.

## Models

### ARX
**With lags 1,2,7**<br>
$p_{d,h} = \beta_{h,0} + \beta_{h,1}p_{d-1,h} + \beta_{h,2}p_{d-2,h} + \beta_{h,3}p_{d-7,h} + \beta_{h,4}p_{d-1}^{min} + \beta_{h,5}load_{d,h} + \beta_{h,6}D_{Sat,h} + \beta_{h,7}D_{Sun,h} + \beta_{h,8}D_{Mon,h} + \beta_{h,9}D_{Holiday,h} + \epsilon_{d,h}$ <br>

**With lags 1,2,...,7**<br>
$p_{d,h} = \beta_{h,0} + \sum_{i=0}^{7} \beta_{h,i}p_{d-i,h} + \beta_{h,8}p_{d-1}^{min} + \beta_{h,9}load_{d,h} + \beta_{h,10}D_{Sat,h} + \beta_{h,11}D_{Sun,h} + \beta_{h,12}D_{Mon,h} + \beta_{h,13}D_{Holiday,h}+ \epsilon_{d,h}$ <br>

### TARX
If mean of yesterday prices is higher than mean of 8 days ago than state 1 else state 0.<br>

**With lags 1,2,7**<br>
$p_{d,h} = \beta_{h,0} +\sum_{j \in {0,1}}( \beta_{h,1,j}p_{d-1,h} + \beta_{h,2,j}p_{d-2,h} + \beta_{h,3,j}p_{d-7,h} + \beta_{h,4,j}p_{d-1}^{min} + \beta_{h,5,j}load_{d,h} + \beta_{h,6,j}D_{Sat,h} + \beta_{h,7,j}D_{Sun,h} + \beta_{h,8,j}D_{Mon,h} + \beta_{h,9,j}D_{Holiday,h} )+ \epsilon_{d,h}$ <br>
for _j_ in {0,1} as state 0 and state 1.<br>

**With lags 1,2,...,7**<br>
$p_{d,h} = \beta_{h,0} + \sum_{j \in {0,1}}( \sum_{i=0}^{7} \beta_{h,i,j}p_{d-l,h} + \beta_{h,8,j}p_{d-1}^{min} + \beta_{h,9,j}load_{d,h} + \beta_{h,10,j}D_{Sat,h} + \beta_{h,11,j}D_{Sun,h} + \beta_{h,12,j}D_{Mon,h} + \beta_{h,13,j}D_{Holiday,h} )+ \epsilon_{d,h}$ <br>
for _j_ in {0,1} as state 0 and state 1.<br>

### mARX
**With lags 1,2,7**<br>
$p_{d,h} = \beta_{h,0} +\beta_{h,1}p_{d-1,h} + \beta_{h,2}p_{d-2,h} + \beta_{h,3}p_{d-7,h} + \beta_{h,4}p_{d-1}^{min} + \beta_{h,5}load_{d,h} + \beta_{h,6}D_{Sat,h} + \beta_{h,7}D_{Sun,h} + \beta_{h,8}D_{Mon,h} + \beta_{h,9}D_{Holiday,h} + \beta_{h,6}D_{Sat,h}p_{d-1,h} + \beta_{h,7}D_{Sun,h}p_{d-1,h} + \beta_{h,8}D_{Mon,h}p_{d-1,h}+ \epsilon_{d,h}$ <br>

**With lags 1,2,...,7**<br>
$p_{d,h} = \beta_{h,0} + \sum_{i=0}^{7} \beta_{h,l}p_{d-i,h} + \beta_{h,8}load_{d,h} + \beta_{h,9}p_{d-1}^{min} + \beta_{h,10}D_{Sat,h} + \beta_{h,11}D_{Sun,h} + \beta_{h,12}D_{Mon,h} + \beta_{h,13}D_{Holiday,h} + \beta_{h,14}D_{Sat,h}p_{d-1,h} + \beta_{h,15}D_{Sun,h}p_{d-1,h} + \beta_{h,16}D_{Mon,h}p_{d-1,h}+ \epsilon_{d,h}$ <br>

## Lasso ## ADD LARS
For each model with lags 1,2,...,7, a regulariazition method called Lasso is employed and insignificant coefficients are forced to be zero in the model. Coefficients of lasso are calculated as such; <br>
$\beta^{Lasso} = \underset{\beta_{h,i}}{\operatorname{argmin}} \left\{ \sum_{d,h} \left( p_{d,h} - \sum_{i}^{n} \beta_{h, i}X_{h,i} \right)^{2} + \lambda \sum_{i}^{n} |\beta_{h,i}| \right\}$ <br>
where $\lambda$ is chosen using point forecasts between period 2019-04-01 and 2020-03-29.

## Forecasts
Point forecasts for each hour are acquired using the last year's values (52\*7 days) as training and one-step ahead forecast is calculated by also taking into account of exogenous variables (load, min price and such.)


Distributional forecasts are calculated in three ways;
- Historical; residuals of the model, i.e. the prediction error in the last year.
- Distributional; random values from a zero-mean normal distibution with std of residuals.
- Bootstrap; a new price series is estimated using the coefficients of the model and a value is forecasted; for 250 times. This process provides a distribution of forecasts (with population of 250).

**Note**: Since the first differenced series are used in the models, the point and distributional forecasts are the differences from the last value. In the forecasting function, the previous value added to point forecast to get the adjusted forecast which is added to distributional forecast to skew the distribution and pile the distribution around the adjusted point forecast.

In [6]:
model_names = [
    'ARX_1_2_7','ARX_to_7', 'ARX_to_7_w_lasso_aic',
    'TARX_1_2_7','TARX_to_7', 'TARX_to_7_w_lasso_aic',
    'mARX_1_2_7','mARX_to_7', 'mARX_to_7_w_lasso_aic',
]

### Point Forecasts

In [24]:
dates_point = ('2019-04-01', '2020-03-29')
p_path = './data/Point_forecasts'
for model in model_names:
    lags = [1,2,7] if '1_2_7' in model else range(1,8)
    model_type = re.match('.*(AR)', model)[0]
    method = 'lasso' if 'lasso' in model else 'ols'
    # construct the model
    globals()[model] = get_forecast_AR(
        main_df,
        price_df_final,
        lags,
        forecast_dates=dates_point,
        model_type=model_type,
        method=method,
        lasso_args={'criterion': 'aic', 'fit_intercept': True, 'normalize': False},
    )
    globals()[model].to_parquet(f'{p_path}/{model}_Point_forecast.parquet')

### Distributional Forecasts

In [27]:
dates_distributional = ('2020-03-30', '2021-03-28')
d_path = './data/Distributional_forecasts'
for model in model_names:
    # adjust parameters
    lags = [1,2,7] if '1_2_7' in model else range(1,8)
    model_type = re.match('.*(AR)', model)[0]
    method = 'lasso' if 'lasso' in model else 'ols'
    # construct the model
    globals()[model + '_dist'] = get_forecast_AR(
        main_df,
        price_df_final,
        lags,
        forecast_dates=dates,
        model_type=model_type,
        method=method,
        lasso_args={'criterion': 'aic', 'fit_intercept': True, 'normalize': False},
        PI_calculate=True,
        bootstrap_B=250,
    )
    globals()[model + '_dist'].to_parquet(f'{d_path}/{file_name}')

## Quantile Regression Averaging
Construct distribution of forecasts using point forecasts of individual models and quantile regression model (Nowotarski & Weron, 2018).

In [29]:
# Construct QRA
qra_range = np.array([0.05, 0.25, 0.5, 0.75, 0.95])
qra_df = pd.concat([
    ARX_1_2_7.Forecast_added.to_frame(name='ARX_1_2_7'),
    ARX_to_7.Forecast_added.to_frame(name='ARX_to_7'),
    ARX_to_7_w_lasso_aic.Forecast_added.to_frame(name='ARX_to_7_w_lasso'),
    TARX_1_2_7.Forecast_added.to_frame(name='TARX_1_2_7'),
    TARX_to_7.Forecast_added.to_frame(name='TARX_to_7'),
    TARX_to_7_w_lasso_aic.Forecast_added.to_frame(name='TARX_to_7_w_lasso'),
    mARX_1_2_7.Forecast_added.to_frame(name='mARX_1_2_7'),
    mARX_to_7.Forecast_added.to_frame(name='mARX_to_7'),
    mARX_to_7_w_lasso_aic.Forecast_added.to_frame(name='mARX_to_7_w_lasso'),
    price_df_final.loc[ARX_1_2_7.index]
], axis=1)

qra_forecast_df = get_forecast_QRA(qra_df, qra_range, dates_distributional)
qra_forecast_df.to_parquet('./data/QRA_forecasts/qra_forecast_df_general.parquet')

In [30]:
# Construct QRA for models 1,2,7
qra_df_1_2_7 = pd.concat([
    ARX_1_2_7.Forecast_added.to_frame(name='ARX_1_2_7'),
    TARX_1_2_7.Forecast_added.to_frame(name='TARX_1_2_7'),
    mARX_1_2_7.Forecast_added.to_frame(name='mARX_1_2_7'),
    price_df_final.loc[ARX_1_2_7.index]
], axis=1)

qra_forecast_df_1_2_7 = get_forecast_QRA(qra_df_1_2_7, qra_range, dates_distributional)
qra_forecast_df_1_2_7.to_parquet('./data/QRA_forecasts/qra_forecast_df_1_2_7.parquet')

In [31]:
# Construct QRA for models to 7
qra_df_to_7 = pd.concat([
    ARX_to_7.Forecast_added.to_frame(name='ARX_to_7'),
    TARX_to_7.Forecast_added.to_frame(name='TARX_to_7'),
    mARX_to_7.Forecast_added.to_frame(name='mARX_to_7'),
    price_df_final.loc[ARX_1_2_7.index]
], axis=1)

qra_forecast_df_to_7 = get_forecast_QRA(qra_df_to_7, qra_range, dates_distributional)
qra_forecast_df_to_7.to_parquet('./data/QRA_forecasts/qra_forecast_df_to_7.parquet')

In [32]:
# Construct QRA models to 7 with lasso
qra_df_to_7_w_lasso = pd.concat([
    ARX_to_7_w_lasso_aic.Forecast_added.to_frame(name='ARX_to_7_w_lasso'),
    TARX_to_7_w_lasso_aic.Forecast_added.to_frame(name='TARX_to_7_w_lasso'),
    mARX_to_7_w_lasso_aic.Forecast_added.to_frame(name='mARX_to_7_w_lasso'),
    price_df_final.loc[ARX_1_2_7.index]
], axis=1)

qra_forecast_df_to_7_w_lasso = get_forecast_QRA(qra_df_to_7_w_lasso, qra_range, dates_distributional)
qra_forecast_df_to_7_w_lasso.to_parquet('./data/QRA_forecasts/qra_forecast_df_to_7_w_lasso.parquet')

## Naive Forecast
Each hour in Tuesday, Wednesday, Thursday and Friday equals to previous day's value, each hour in Monday, Saturday and Sunday is the same as last week's value (Nowotarski & Weron, 2018). Distributional forecasts are calculated using the error of point forecast in the last year (subsequent 52*7 days).

In [ ]:
# NAIVE forecast for total data
naive_forecast_df, auto_ar_forecast_df = get_naive_forecast(price_df_final, forecast_dates=('2020-03-30', '2021-03-28'))
naive_forecast_df.to_parquet('./data/Benchmark_forecasts/Naive_forecast_df.parquet')
auto_ar_forecast_df.to_parquet('./data/Benchmark_forecasts/AutoArima_forecast_df.parquet')

## Sample Plot

In [ ]:
# fig_df = pd.concat([
#     qra_df.loc[sample_week[0]:],
#     qra_forecast_df['0.05'].to_frame('QRA Lower'),
#     qra_forecast_df['0.5'].to_frame('QRA Median'),
#     qra_forecast_df['0.95'].to_frame('QRA Upper'),
#     naive_forecast_df.loc[sample_week[0]:sample_week[1]].Naive_forecast
# ],axis=1
# )

# fig = go.Figure()
# for col in fig_df:
#     fig.add_trace(go.Scatter(y=fig_df[col], x=fig_df.index, name=col))
# fig.update_layout(
#     hovermode="x",
#     title=f'Sample Plot for {sample_week[0]} to {sample_week[1]}'
# )
# fig.show('png')
# # fig.show() to get the interactive plot

## Get Prediction Intervals and combinations
This section grabs the distributional forecast of each model, combines them by utilizing various methods to get a lower and a upper bound following Gaba et al. (2017);
- Mean; simple average of lower and upper bounds of each model included.
- Median; median value of lower and upper bounds of each model included.
- Envelope; minimum of lower bounds and maximum of upper bounds.
- Interior trimming; excludes upper $\beta$ percent of the lower bounds and lower $\beta$ percent of upper bounds and takes a simple average. (Wider than simple average given $1>\beta>0$)
- Exterior trimming; excludes lower $\beta$ percent of the lower bounds and upper $\beta$ percent of upper bounds and takes a simple average. (Narrower than simple average given $1>\beta>0$)

In [66]:
for model in model_names:
    globals()["PI_"+model] = get_PI_percentiles(eval(model))

In [82]:
# Calculate simple combinations
PI_simple_combinations_df = PI_combinations(
    models=[
        PI_ARX_1_2_7.PI_historical,
        PI_ARX_1_2_7.PI_distributional,
        PI_ARX_1_2_7.PI_bootstrap,
        PI_ARX_to_7.PI_historical,
        PI_ARX_to_7.PI_distributional,
        PI_ARX_to_7.PI_bootstrap,
        PI_ARX_to_7_w_lasso_aic.PI_historical,
        PI_ARX_to_7_w_lasso_aic.PI_distributional,
        PI_ARX_to_7_w_lasso_aic.PI_bootstrap,
        PI_TARX_1_2_7.PI_historical,
        PI_TARX_1_2_7.PI_distributional,
        PI_TARX_to_7.PI_historical,
        PI_TARX_to_7.PI_distributional,
        PI_TARX_to_7_w_lasso_aic.PI_historical,
        PI_TARX_to_7_w_lasso_aic.PI_distributional,
        PI_mARX_1_2_7.PI_historical,
        PI_mARX_1_2_7.PI_distributional,
        PI_mARX_1_2_7.PI_bootstrap,
        PI_mARX_to_7.PI_historical,
        PI_mARX_to_7.PI_distributional,
        PI_mARX_to_7.PI_bootstrap,
        PI_mARX_to_7_w_lasso_aic.PI_historical,
        PI_mARX_to_7_w_lasso_aic.PI_distributional,
        PI_mARX_to_7_w_lasso_aic.PI_bootstrap,
    ],
    PI = [0.5, 0.9],
    beta = 0.4 # for trimming
)

In [ ]:
# Calculate IC based combination
weights_dict = {x: eval(x).model_aic for x in model_names}
weights_df = pd.concat(weights_dict.values(), axis=1, keys=weights_dict.keys()) \
.dropna() \
.apply(lambda x: x.apply(lambda y: y - min(x)), axis=1) \
.apply(lambda x: x.apply(lambda y: np.exp((-1/2)*y) / sum(np.exp((-1/2)*x))), axis=1)

get_IC_based_combination = lambda PI_type: pd.concat(
    [eval('PI_'+ x)[PI_type] for x in model_names],
    axis=1,
    keys=model_names
) \
.groupby(axis=1, level=1) \
.apply(lambda g: g.dropna().multiply(weights_df.values).sum(axis=1))

PI_IC_combinations_historical = get_IC_based_combination('PI_historical')
PI_IC_combinations_distributional = get_IC_based_combination('PI_distributional')
PI_IC_combinations_bootstrap = get_IC_based_combination('PI_bootstrap')

## Reliability

#### Get percentiles of naive distributional forecast
Since the negative prices are not allowed in Turkish market, negative values are set to 0.

In [ ]:
percentile_df_naive = get_PI_from_distribution(
    naive_forecast_df.PI_historical,
    return_negative_values=False,
    percentiles=np.array([5, 25, 50, 75, 95])
)

### Unconditional and Conditional Coverage
#### Showcase using naive forecast
Unconditional coverage likelihood ratio (i.e. Kupiec's proportion of failures) test and Conditional Coverage likelihood ratio test (Nowotarski & Weron, 2018; Christoffersen, 1998)

In [114]:
lags = [1,2,7]# [1,2,7]
def plot_coverage(percentile_df, price_df, lags, title):
    
    Coverage_LR_Scores_50 = Christoffersen_scores(
        percentile_df=percentile_df.dropna(),
        realized=price_df,
        PI=0.5,
        lags=lags
    )
    Coverage_LR_Scores_90 = Christoffersen_scores(
        percentile_df=percentile_df,
        realized=price_df,
        PI=0.9,
        lags=lags
    )

    fig, ax= plt.subplots(len(lags)+1, figsize=(10,10))
    ax[0].set_title(f'Unconditional Coverage - {title}')
    Coverage_LR_Scores_90.UC_LR.apply(lambda x: 20 if x>20 else x).reset_index().plot(
        x='index', y='UC_LR', kind='scatter', ax=ax[0], color='blue', label='90% PI'
    )
    Coverage_LR_Scores_50.UC_LR.apply(lambda x: 20 if x>20 else x).reset_index().plot(
        x='index', y='UC_LR', kind='scatter', ax=ax[0], color='red', label='50% PI', xlabel='hour'
    )
    ax[0].axhline(y=stats.chi2.ppf(0.99,df=1))
    ax[0].axhline(y=stats.chi2.ppf(0.95,df=1), ls='--')

    for i, l in zip(range(1,len(lags)+1), lags):
        ax[i].set_title(f'Conditional Coverage - {title} ({l} day lag)')
        Coverage_LR_Scores_90[f'CC_LR_lag{l}'].apply(lambda x: 20 if x>20 else x).reset_index().plot(
            x='index', y=f'CC_LR_lag{l}', kind='scatter', ax=ax[i], color='blue', label='90% PI'
        )
        Coverage_LR_Scores_50[f'CC_LR_lag{l}'].apply(lambda x: 20 if x>20 else x).reset_index().plot(
            x='index', y=f'CC_LR_lag{l}', kind='scatter', ax=ax[i], color='red', label='50% PI',  xlabel='hour'
        )
        ax[i].axhline(y=stats.chi2.ppf(0.99,df=2))
        ax[i].axhline(y=stats.chi2.ppf(0.95,df=2), ls='--')

    fig.tight_layout()
    return fig

In [116]:
# plot_coverage(PI_ARX_1_2_7.PI_historical, price_df_final, lags=[1,2,7], title='ARX')

# Sharpness

## Winkler Score

In [ ]:
### Naive
winkler_naive_90 = winkler_score(percentile_df_naive, price_df_final, PI=0.9)
winkler_naive_50 = winkler_score(percentile_df_naive, price_df_final, PI=0.5)
### ARX 1,2,7
winkler_ARX127_90 = winkler_score(get_PI_wrapper_csv(ARX_1_2_7.dropna()).PI_bootstrap, price_df_final, PI=0.9)
winkler_ARX127_50 = winkler_score(get_PI_wrapper_csv(ARX_1_2_7.dropna()).PI_bootstrap, price_df_final, PI=0.5)
# QRA
winkler_qra_90 = winkler_score(qra_forecast_df, price_df_final, PI=0.9)
winkler_qra_50 = winkler_score(qra_forecast_df, price_df_final, PI=0.5)
### PI combinations
winkler_PIcombinations_90_df = PI_combinations_df.groupby(level=0, axis=1).apply(
    lambda method: winkler_score(method.droplevel(0,1), price_df_final, PI=0.9)
)
winkler_PIcombinations_50_df = PI_combinations_df.groupby(level=0, axis=1).apply(
    lambda method: winkler_score(method.droplevel(0,axis=1), price_df_final, PI=0.5)
)

## Diebold - Mariano test
Test whether the difference in winkler scores is significant for two competing models.

In [ ]:
# 90% PI
DM_winkler_ARX127_vs_naive_90 = DM_test(winkler_ARX127_90.Winkler_score, winkler_naive_90.Winkler_score, alpha=0.1, two_sided=True)
DM_winkler_PIcomb_vs_ARX127_90 = winkler_PIcombinations_90_df.groupby(level=0, axis=1).apply(
    lambda method: DM_test(method.droplevel(0,1).Winkler_score, winkler_ARX127_90.Winkler_score, alpha=0.1, two_sided=True)
)
DM_winkler_PIcomb_vs_naive_90 = winkler_PIcombinations_90_df.groupby(level=0, axis=1).apply(
    lambda method: DM_test(method.droplevel(0,1).Winkler_score, winkler_naive_90.Winkler_score, alpha=0.1, two_sided=True)
)
DM_winkler_PIcomb_vs_qra_90 = winkler_PIcombinations_90_df.groupby(level=0, axis=1).apply(
    lambda method: DM_test(method.droplevel(0,1).Winkler_score, winkler_qra_90.Winkler_score, alpha=0.1, two_sided=True)
)
# 50% PI
DM_winkler_ARX127_vs_naive_50 = DM_test(winkler_ARX127_50.Winkler_score, winkler_naive_50.Winkler_score, alpha=0.1, two_sided=True)
DM_winkler_PIcomb_vs_ARX127_50 = winkler_PIcombinations_50_df.groupby(level=0, axis=1).apply(
    lambda method: DM_test(method.droplevel(0,1).Winkler_score, winkler_ARX127_50.Winkler_score, alpha=0.1, two_sided=True)
)
DM_winkler_PIcomb_vs_naive_50 = winkler_PIcombinations_50_df.groupby(level=0, axis=1).apply(
    lambda method: DM_test(method.droplevel(0,1).Winkler_score, winkler_naive_50.Winkler_score, alpha=0.1, two_sided=True)
)
DM_winkler_PIcomb_vs_qra_50 = winkler_PIcombinations_50_df.groupby(level=0, axis=1).apply(
    lambda method: DM_test(method.droplevel(0,1).Winkler_score, winkler_qra_50.Winkler_score, alpha=0.1, two_sided=True)
)

In [ ]:
DM_winkler_PIcomb_vs_naive_50